In [12]:
# Installation
!pip install -q youtube-transcript-api langchain-community faiss-cpu

In [ ]:
from dotenv import load_dotenv
import os

# 1. Load environment variables
load_dotenv()

# 2. Access your Hugging Face token
hf_token = os.getenv("HF_TOKEN")

In [14]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate

Step1a-Indexing(Document Ingestion)

In [15]:
# video_id="6Ax8IP0Vt5Q"
# try:
#      # If you don’t care which language, this returns the “best” one
#     transcript_list=YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
#     # print(transcript_list)
#     # print(transcript_list[0]['text'])
    
#     # Flattening into plan text
#     trancript="".join(chunk["text"] for chunk in transcript_list)
#     print(trancript)
    
# except TranscriptsDisabled:
#     print("No Caption available for this video")

In [16]:
video_id = "6Ax8IP0Vt5Q"
try:
    ytt_api = YouTubeTranscriptApi()
    fetched_transcript = ytt_api.fetch(video_id, languages=['en'])  # Returns a FetchedTranscript object

    # Convert to raw data if needed
    transcript_list = fetched_transcript.to_raw_data()

    # print(transcript_list)
    # print(transcript_list['text'])

    # Flatten into plain text
    trancript = "".join(chunk['text'] for chunk in transcript_list)
    print(trancript)

except TranscriptsDisabled:
    print("No Caption available for this video")

Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh almost anyembedded developer uh using C languagefor programming these tiny devices. Uhbut soon enough I figured out that C++can be useful for uh what I'm doing andI started applying it and us

In [17]:
trancript

"Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh almost anyembedded developer uh using C languagefor programming these tiny devices. Uhbut soon enough I figured out that C++can be useful for uh what I'm doing andI started applying it and u

Step 1b - Indexing(Text Splitting)

In [18]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
chunks=splitter.create_documents([trancript])
chunks

[Document(metadata={}, page_content="Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh almost anyembedded developer uh using C languagefor programming these tiny devices. Uhbut soon enough I figured out that C++can be useful for uh what I'm 

In [19]:
len(chunks)

45

In [20]:
chunks[5]

Document(metadata={}, page_content="provide you withC++ 17 right and the book was written uhmostly last year. So um that's uh anissue for people that are working on uhcritical on on critical systems and yeahthey are basically bound to using uh nowthe C++ 17 um if they want to complywith the regulations. All right. uhstill uh if you're not working onsomething that's safety critical you canuse uh GCC supports the latest standardsand hand as well so you can use thoseright uh beside this uh we also brieflydiscussed this other useful things suchas static analyzersum ufor unit testing and a bit profilingprofiling is interesting in the contextof the embedded development as uh youcan't really use the tools that you canon your desk machines right because youneed to interact with embedded targetstill there's plenty of options uh forthere that as well and uh I'm suggestingjust a few uh that might be useful forpeople it's not really specific to C++it's just you know in general whetheryou're writin

Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [21]:
# Calling the hf embedding model
embedding = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-mpnet-base-v2",
    task="feature-extraction",
    huggingfacehub_api_token=hf_token
)
# Stroing the Vectore store
vector_store=FAISS.from_documents(chunks,embedding)


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/sentence-transformers/all-mpnet-base-v2?expand=inferenceProviderMapping (Request ID: Root=1-68b41812-0044334325df77d564c999ae;3eb78c6b-bf73-424e-a912-e9e4b5c40ed0)

Invalid credentials in Authorization header

In [ ]:
vector_store.index_to_docstore_id

{0: 'ce312962-1c28-4549-a3a6-b0f8556f8ac9',
 1: 'fcc1b040-f75c-4660-94fa-dee32ca2c032',
 2: '5c726aaf-2b1d-4a68-964f-aba0526604bf',
 3: '2404d283-1640-4c99-95b8-47a8b4c15e62',
 4: '9f019878-bed0-4c26-9b06-20a5202303b4',
 5: '77a220f8-5a05-4d51-8abf-62e8ec85a3e3',
 6: 'c8edd823-df19-4dd1-baf9-9e388b42efae',
 7: '16cac4d6-89ef-45bf-a0e7-69cb5acd7dae',
 8: '7c1ed786-82fb-48ca-a5b1-58ed2eab11f7',
 9: 'dd83e1be-7a1b-4009-a2b5-00fa12a17a6e',
 10: '4cdcc4cf-05a0-407a-bb58-9d4248c88d2a',
 11: '30d8c02a-0699-4aaa-b897-e76c959d0939',
 12: 'fec9c58a-430e-4abe-9ef9-16223ce0831d',
 13: 'af5d3549-40f9-43aa-9db0-828c0b470f8d',
 14: '439c9312-655e-490a-a3bf-a6a8f3642390',
 15: '418d80e0-4b99-4576-87b2-44a82e6cd81c',
 16: 'f5fcb9b8-2359-467f-b947-de399f74406e',
 17: '53f441b5-9370-4ee6-b09a-f3ca34aea5b2',
 18: '1756d062-845d-459c-a04c-2a7b54f02457',
 19: '6ec43fda-8629-4818-bca6-033810839f1e',
 20: '086fd62b-fe3d-42d7-8f3d-831d742466b1',
 21: '148fb8e4-baa4-4c79-9d7c-d8109dde572b',
 22: 'eb1bf81c-59aa-

In [ ]:
vector_store.get_by_ids(['1b97580d-d069-4862-a4b5-5128aefd98a2'])

[]

Step 2 - Retrieval

In [ ]:
retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={"k":4}
)
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026AD2BA3DC0>, search_kwargs={'k': 4})

In [ ]:
# retriever.invoke('What is deepmind in AI')

[Document(id='190bba8f-a401-4f06-ab74-d40f21083e96', metadata={}, page_content="function whichare not using dynamic memory allocationuh which is something that's importantfor lot of embedded people so I use thatopportunity to introduce it and to showit how how it can be used right. Uh theother thing we we do there isintroducing uh stood priority Q and weprovide it with I believe uh stood uhnot stood vector but ETL vector and uhETL vector is vector from em embeddedtemplate library. It has the sameinterface as stood vector. The onlydifference is that uh once youinitialize it you give it size. It hasfixed size. So you can think about it asa fixed size vector which is alsoimportant for embedded people as they'regoing to be certain that there's nodynamic memory allocation and suchright.>> Yeah.Yeah. That's interesting. And especiallylike there's a domain specific libraryum which also kind of you know has typeswhich are similar to the modern C++focus>> and that that allow you that allows you

Step 3 - Augmentation

"""
Augumentation referts to the steps where the retieved document (chunks of relevant context) are combined with the user's query
to for a new, enriched prompt for the LLM

{most relevent chunk) +(Query) = Prompt
"""

In [ ]:
# Creating combined prompt
prompt = PromptTemplate(
    template="""
    You are the helpful assistant.
    Answer ONLY from the provided transcript context
    If the context is the insufficient , just say you don't know
    {context}
    Question:{question}
    """,
    input_variables=['context','question']
)


In [ ]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

[Document(id='ce312962-1c28-4549-a3a6-b0f8556f8ac9', metadata={}, page_content="Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.>> Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh almost anyembedded developer uh using C languagefor programming these tiny devices. Uhbut soon enough I figu

In [ ]:
# Creating Context from teh retrieved_docus
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.>> Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh almost anyembedded developer uh using C languagefor programming these tiny devices. Uhbut soon enough I figured out that C++can be useful for uh what I'm doing andI started applying it an

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
final_prompt

StringPromptValue(text="\n    You are the helpful assistant.\n    Answer ONLY from the provided transcript context\n    If the context is the insufficient , just say you don't know\n    Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.>> Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh al

Step 4 - Generation

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation",
    huggingfacehub_api_token=hf_token  # <--- PASS YOUR TOKEN HERE
)

# 4. Initialize ChatHuggingFace with the llm
model = ChatHuggingFace(llm=llm)

# answer=model.invoke(final_prompt)
# print(answer.content)

Step 5 - Building Chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('who is Demis')

{'context': "Hi and welcome to today's meeting C++interview with Amar who's written a bookon embedded with C++. Uh let me quicklyshow you the book. This is a book whichwe're going to see in a minute. Um butfirst let's get started with anintroduction to know actually who Amaris.>> Um yeah uh hi Yens. Hi everyone. Uh Yensthank you for having me. I'm very muchexcited to be here. to be able uh totalk about my book. And I'd like toapologize to everyone for the delay. Uhwe had technical difficultiesuh with this Bluetooth technology. Butyeah, I hope I'm coming over well andyou can hear me well. And yeah, I'm anembedded engineer for uh almost 10 yearsnow. And uh around seven years ago Istarted a small company in Bostonhergoina called assembly and we do a lotof interesting uh embedded projects. UmI started my career as uh almost anyembedded developer uh using C languagefor programming these tiny devices. Uhbut soon enough I figured out that C++can be useful for uh what I'm doing andI started ap

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Can you summarize the video')

ValueError: Model google/gemma-2-2b-it is not supported for task text-generation and provider nebius. Supported task: conversational.